In [1]:
import chromedriver_binary
from selenium import webdriver
from time import sleep
import openpyxl
import pandas as pd
from sucreiping import Sucreiping

In [2]:
sucreiping = Sucreiping()

In [3]:
sucreiping.get_year()

['https://www.kenchikukyo.jp/schedule/year/2022/',
 'https://www.kenchikukyo.jp/schedule/year/2021/',
 'https://www.kenchikukyo.jp/schedule/year/2020/']

In [5]:
df = sucreiping.get_data()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=61554): Max retries exceeded with url: /session/7dbc5df83b85f10ee07ea708feeeb436/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f88e9aee340>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [7]:
column = df.columns[5:]

for col in column:
    df[col] = df[col].astype('int')

In [8]:
df_number = df[["日令","体重","価格"]]
corr_matrix = df_number.corr().abs()
corr_matrix.head()

1,日令,体重,価格
1,,,
日令,1.000000,0.067028,0.078040
体重,0.067028,1.000000,0.594039
価格,0.078040,0.594039,1.000000


In [9]:
df_dummied = pd.get_dummies(df)
df_dummied

,日令,体重,価格,性別_去勢,性別_雄,性別_雌,父牛_久富福,父牛_久晴茂,父牛_久百合,父牛_久福久,...,母の祖祖父_雲仙丸,母の祖祖父_頼山,母の祖祖父_飛騨白清,母の祖祖父_高栄,母の祖祖父_高森,母の祖祖父_高竜,母の祖祖父_鶴長,母の祖祖父_鶴１６５の９,母の祖祖父_鹿秀土井,母の祖祖父_２１世紀
0,292,276,618200,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,286,293,782100,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,297,305,697400,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,297,258,676500,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,297,301,712800,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28226,294,314,748000,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28227,290,318,806300,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28228,259,309,871200,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28229,293,353,875600,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df.head()

1,性別,父牛,母の父,母の祖父,母の祖祖父,日令,体重,価格
0,去勢,愛之国,安福（岐）,平茂勝,金幸,292,276,618200
1,去勢,愛之国,安福久,平茂勝,安平,286,293,782100
2,雌,愛之国,華春福,安福久,忠茂勝,297,305,697400
3,雌,愛之国,美国桜,美津照重,勝平正,297,258,676500
4,去勢,愛之国,百合勝安,安福勝,寿恵福,297,301,712800


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28231 entries, 0 to 28230
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   性別      28231 non-null  object
 1   父牛      28231 non-null  object
 2   母の父     28231 non-null  object
 3   母の祖父    28231 non-null  object
 4   母の祖祖父   28231 non-null  object
 5   日令      28231 non-null  int64 
 6   体重      28231 non-null  int64 
 7   価格      28231 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 1.7+ MB


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [12]:
X = df_dummied.drop(columns = ['価格'])
y = df_dummied['価格']

train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [18]:
#  RandomForest回帰
model_1 = RandomForestRegressor(100)  # ランダムフォレスト回帰モデル
model_1.fit(train_X, train_y)  # 学習
pred_y = model_1.predict(test_X)  # 予測
mse = mean_squared_error(test_y, pred_y)  #評価
print(mse)
print("正答率（学習データ）:", model_1.score(train_X, train_y))
print("正答率（テストデータ）:", model_1.score(test_X, test_y))

12971160806.495926
正答率（学習データ）: 0.9340734262799628
正答率（テストデータ）: 0.5163688514966864


In [14]:
model_2 = Ridge(alpha=0.1).fit(train_X, train_y)
pred_y = model_2.predict(test_X)  # 予測
mse = mean_squared_error(test_y, pred_y)  #評価
print(mse)
print("正答率（学習データ）:", model_2.score(train_X, train_y))
print("正答率（テストデータ）:", model_2.score(test_X, test_y))

13894608828.262793
正答率（学習データ）: 0.5473927132295132
正答率（テストデータ）: 0.48193799106617174


In [15]:
model_3 = Lasso(alpha=0.1).fit(train_X, train_y) 
pred_y = model_3.predict(test_X)  # 予測
mse = mean_squared_error(test_y, pred_y)  #評価
print(mse)
print("正答率（学習データ）:", model_3.score(train_X, train_y))
print("正答率（テストデータ）:", model_3.score(test_X, test_y))

13964793776.20549
正答率（学習データ）: 0.5474807222508968
正答率（テストデータ）: 0.47932113761045225


/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.028e+14, tolerance: 5.132e+10
  model = cd_fast.enet_coordinate_descent(


In [16]:
dtrain = xgb.DMatrix(train_X, train_y)
dvalid = xgb.DMatrix(test_X, test_y)
params = {
    "objective" : "reg:squarederror",
    "eval_metric" : "rmse"
}

model = xgb.train(
    params = params,
    dtrain = dtrain,
    evals = [(dtrain, "train"), (dvalid, "valid")],
    num_boost_round = 100
)

pred = model.predict(xgb.DMatrix(test_X))

[0]	train-rmse:528094.51203	valid-rmse:529920.99056
[1]	train-rmse:378860.01786	valid-rmse:381076.97973
[2]	train-rmse:277045.37121	valid-rmse:279668.83490
[3]	train-rmse:209191.72429	valid-rmse:212354.26827
[4]	train-rmse:165607.34784	valid-rmse:169806.68272
[5]	train-rmse:139005.22451	valid-rmse:144312.00427
[6]	train-rmse:123171.10401	valid-rmse:129508.31561
[7]	train-rmse:114290.37664	valid-rmse:121432.37966
[8]	train-rmse:109382.51483	valid-rmse:117171.09307
[9]	train-rmse:106466.44368	valid-rmse:114857.12819
[10]	train-rmse:104736.26766	valid-rmse:113498.13258
[11]	train-rmse:103677.97118	valid-rmse:112777.66148
[12]	train-rmse:102940.93485	valid-rmse:112235.74353
[13]	train-rmse:102500.13641	valid-rmse:112008.58159
[14]	train-rmse:101949.60277	valid-rmse:111758.38538
[15]	train-rmse:101454.52160	valid-rmse:111565.36749
[16]	train-rmse:100677.38873	valid-rmse:110716.98672
[17]	train-rmse:100352.63746	valid-rmse:110436.99330
[18]	train-rmse:100061.99636	valid-rmse:110286.08557
[19

In [17]:
df.head()

1,性別,父牛,母の父,母の祖父,母の祖祖父,日令,体重,価格
0,去勢,愛之国,安福（岐）,平茂勝,金幸,292,276,618200
1,去勢,愛之国,安福久,平茂勝,安平,286,293,782100
2,雌,愛之国,華春福,安福久,忠茂勝,297,305,697400
3,雌,愛之国,美国桜,美津照重,勝平正,297,258,676500
4,去勢,愛之国,百合勝安,安福勝,寿恵福,297,301,712800


In [16]:
sex = input("性別を入力してください:")
father = input("1代祖を入力してください:")
gland = input("2代祖を入力してください:")
gege = input("3代祖を入力してください:")
got = input("4代祖を入力してください:")
age = input("日齢を入力してください:")
wight = input("体重を入力してください:")

df = df.append({'性別': sex, '父牛': father, '母の父': gland, '母の祖父': gege, '母の祖祖父': got, '日令': age, '体重': wight, '価格': 1}, ignore_index=True)

性別を入力してください: 雌
1代祖を入力してください: 愛之国
2代祖を入力してください: 華春福
3代祖を入力してください: 安福久
4代祖を入力してください: 忠茂勝
日齢を入力してください: ２９７
体重を入力してください: ３０５


/var/folders/5w/f6yfy31s1fldqfnxdzv8ycj00000gn/T/ipykernel_13749/3585564614.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'性別': sex, '父牛': father, '母の父': gland, '母の祖父': gege, '母の祖祖父': got, '日令': age, '体重': wight, '価格': 1}, ignore_index=True)


In [17]:
df.tail()

,性別,父牛,母の父,母の祖父,母の祖祖父,日令,体重,価格
26166,雌,諒太郎,福之国,福桜（宮）,安平,290,318,806300
26167,去勢,諒太郎,平茂晴,安糸福,北国７の８,259,309,871200
26168,去勢,諒太郎,隆之国,安福久,平茂勝,293,353,875600
26169,雌,鈴音,美津照重,第１花国,紋次郎,288,254,689700
26170,雌,愛之国,華春福,安福久,忠茂勝,２９７,３０５,1


In [18]:
def price_predict(df):
    column = df.columns[5:]

    for col in column:
        df[col] = df[col].astype('int')

    df_dummied = pd.get_dummies(df)

    e = df_dummied.tail(1)

    a = e.drop(columns = ['価格'])
    b = e['価格']
    d = model.predict(xgb.DMatrix(a)) 
    df = df[:-1]
    return int(d),df

In [19]:
price_predict(df)[0]

687646

In [21]:
df = price_predict(df)[1]

In [22]:
df.tail()

,性別,父牛,母の父,母の祖父,母の祖祖父,日令,体重,価格
26165,雌,諒太郎,福栄,平茂勝,安平,294,314,748000
26166,雌,諒太郎,福之国,福桜（宮）,安平,290,318,806300
26167,去勢,諒太郎,平茂晴,安糸福,北国７の８,259,309,871200
26168,去勢,諒太郎,隆之国,安福久,平茂勝,293,353,875600
26169,雌,鈴音,美津照重,第１花国,紋次郎,288,254,689700
